# Labs organized for Castor upload

### This lab layout follows the request of scientists and Castor staff  for upload in Castor

In [1]:
#import libraries

In [2]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

sys.path.insert(0, '../../')
from brickstudy import foundation

In [3]:
### import example lab organization from scientst

In [4]:
example = pd.read_csv('../../secret_data/Lab_vb_makeda.csv', sep=";")
example.head(4)

,Participant Id,Participant Status,Site Abbreviation,Participant Creation Date,ERY0,HB0,MCV0,HT0,RDW0,RETI0,...,ASAT0,AFOS0,GGT0,CRP0,KREA_U0,TE_U0,TE_U_KR0,ALB_U0,ALB_U_KR0,B120
0,BRICK_001,Enrolled,ERA,12-4-2024 11:39,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
example.columns

Index(['Participant Id', 'Participant Status', 'Site Abbreviation',
       'Participant Creation Date', 'ERY0', 'HB0', 'MCV0', 'HT0', 'RDW0',
       'RETI0', 'LEU0', 'TROMBO0', 'FE0', 'FERT0', 'TRAF0', 'TSAT0', 'ALAT0',
       'LDH0', 'TBIL0', 'DBIL0', 'KREA0', 'SCHWARTZ_Bedsite0', 'NRCA', 'GRAT0',
       'FOLZ0', 'UREU0', 'NA0', 'K00', 'VITD0', 'ASAT0', 'AFOS0', 'GGT0',
       'CRP0', 'KREA_U0', 'TE_U0', 'TE_U_KR0', 'ALB_U0', 'ALB_U_KR0', 'B120'],
      dtype='object')

In [6]:
example.shape

(1, 39)

In [7]:
# I reccomend turning K00 into K0, and NRCA to NRC0

### import baseline labs from 2022 and 2023
These labs should be named with _01 and uploaded to Castor as repeating data

In [8]:
#lab_data_baseline = pd.read_excel('../../secret_data/Brick_lab_20230914.xlsx')
lab_data = pd.read_csv('../../secret_data/BRICK_lab_data_update_18_03_2024.csv', sep= ';', encoding='latin1')

In [57]:
lab_data_baseline.head(3)

,studieID,effectiveDateTime,issued,code_code_original,code_display_original,hix_code,valueQuantity_comparator,valueQuantity_value,valueQuantity_unit,valueString,comment,interpretation_display,referenceRange_text,specimen_type,specimen_type_text,container_type,sample_note
0,BRICK-001,2022-06-20 15:09:28,2022-06-20 16:04:08,ORD8,Order(ORD1) - test bepaling tbv declaratie,NaN,NaN,NaN,NaN,DECLARATIE,NaN,NaN,NaN,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
1,BRICK-001,2022-06-20 15:09:28,2022-06-20 16:04:08,ALB_U,Albumine in urine,M-albu.,NaN,0.008,g/L,NaN,NaN,Normal,< 0.05,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
2,BRICK-001,2022-06-20 15:09:28,2022-06-20 16:04:08,TE_U,Tot. eiwit in urine,Tot.eiwit,NaN,0.070,g/l,NaN,NaN,Normal,< 0.15,Urine,Urine midstream,Urinebuis 6.0 ml,NaN


In [58]:
len(lab_data_baseline.studieID.unique())

48

In [59]:
lab_data_baseline['studieID'] = lab_data_baseline['studieID'].astype(str)
lab_data_baseline['studieID'] = lab_data_baseline['studieID'].str.replace("-", "_")

In [60]:
lab_data_baseline.head(3)

,studieID,effectiveDateTime,issued,code_code_original,code_display_original,hix_code,valueQuantity_comparator,valueQuantity_value,valueQuantity_unit,valueString,comment,interpretation_display,referenceRange_text,specimen_type,specimen_type_text,container_type,sample_note
0,BRICK_001,2022-06-20 15:09:28,2022-06-20 16:04:08,ORD8,Order(ORD1) - test bepaling tbv declaratie,NaN,NaN,NaN,NaN,DECLARATIE,NaN,NaN,NaN,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
1,BRICK_001,2022-06-20 15:09:28,2022-06-20 16:04:08,ALB_U,Albumine in urine,M-albu.,NaN,0.008,g/L,NaN,NaN,Normal,< 0.05,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
2,BRICK_001,2022-06-20 15:09:28,2022-06-20 16:04:08,TE_U,Tot. eiwit in urine,Tot.eiwit,NaN,0.070,g/l,NaN,NaN,Normal,< 0.15,Urine,Urine midstream,Urinebuis 6.0 ml,NaN


### We want  all the patients on thier own line, but not many lines per patient
We will try to accomplish this by splitting the dataset into each code-code-original i.e. each type of lab taken on the assumption we will almost always have one lab

In [61]:
lab_data_baseline.code_code_original.unique()

array(['ORD8', 'ALB_U', 'TE_U', 'TE_U_KR', 'ALB_U_KR', 'KREA_U', 'LORCCA',
       'BRICK', 'LDH', 'SCHWARTZ-Bedsite', 'LENGTH_ORD', 'LENGTH_DAT',
       'TRAF', 'KREA', 'CRP', 'DBIL', 'FERT', 'B12', 'FOLZ', 'K', nan,
       'FE', 'BAF9', 'UREU', 'TBIL', 'AFOS', 'GGT', 'ASAT', 'ALAT',
       'VITD', 'MCV', 'RDW', 'GRAT', 'HB', 'HT', 'LEU', 'ERY', 'TROMBO',
       'RETI', 'NRCA', 'ICT', 'LIP', 'HEM', 'TSAT', 'TNFa', 'OSMO_U',
       'GLUC_AO', 'SGEW', 'Eiwit', 'ERY_UR', 'LEUKO', 'Ketonen',
       'Bilirubine', 'Urobilinogeen', 'Nitriet', 'PH', 'UREU_U',
       'AB12_Roche', 'BGL', 'SCR', 'KRBL', 'G6PD', 'PIKI', 'F351',
       'FLAM_opti', 'FKAP_opti', 'FKALA_UR', 'BGK', 'HPLC HbA2',
       'HPLC HbF', 'HPLC HbS', 'BASO_DA', 'EO_DA', 'NEUT_DA', 'LYMFO_DA',
       'MONO_DA', 'HPLC HbA0', 'eDA_Neutro#', 'eDA_IG#', 'eDA_Baso#',
       'eDA_Eo#', 'eDA_Lymfo#', 'eDA_Mono#', 'IG_DA', 'ENDO_dummy', 'PT',
       'INR', 'FBGC', 'APTT', 'CAC', 'CAG', 'SVRIJ', 'SAC', 'STOT', 'F8S',
       'ATRO', 'P

In [62]:
codes = lab_data_baseline.code_code_original.unique().astype(str)
codes

array(['ORD8', 'ALB_U', 'TE_U', 'TE_U_KR', 'ALB_U_KR', 'KREA_U', 'LORCCA',
       'BRICK', 'LDH', 'SCHWARTZ-Bedsite', 'LENGTH_ORD', 'LENGTH_DAT',
       'TRAF', 'KREA', 'CRP', 'DBIL', 'FERT', 'B12', 'FOLZ', 'K', 'nan',
       'FE', 'BAF9', 'UREU', 'TBIL', 'AFOS', 'GGT', 'ASAT', 'ALAT',
       'VITD', 'MCV', 'RDW', 'GRAT', 'HB', 'HT', 'LEU', 'ERY', 'TROMBO',
       'RETI', 'NRCA', 'ICT', 'LIP', 'HEM', 'TSAT', 'TNFa', 'OSMO_U',
       'GLUC_AO', 'SGEW', 'Eiwit', 'ERY_UR', 'LEUKO', 'Ketonen',
       'Bilirubine', 'Urobilinogeen', 'Nitriet', 'PH', 'UREU_U',
       'AB12_Roche', 'BGL', 'SCR', 'KRBL', 'G6PD', 'PIKI', 'F351',
       'FLAM_opti', 'FKAP_opti', 'FKALA_UR', 'BGK', 'HPLC HbA2',
       'HPLC HbF', 'HPLC HbS', 'BASO_DA', 'EO_DA', 'NEUT_DA', 'LYMFO_DA',
       'MONO_DA', 'HPLC HbA0', 'eDA_Neutro#', 'eDA_IG#', 'eDA_Baso#',
       'eDA_Eo#', 'eDA_Lymfo#', 'eDA_Mono#', 'IG_DA', 'ENDO_dummy', 'PT',
       'INR', 'FBGC', 'APTT', 'CAC', 'CAG', 'SVRIJ', 'SAC', 'STOT', 'F8S',
       'ATRO', 

In [35]:
#Here we must decide wif we want to switch nan to NA, or we will have no NA0 value in the end

In [74]:
lab_data_baseline[lab_data_baseline['code_code_original'].isna()]

,studieID,effectiveDateTime,issued,code_code_original,code_display_original,hix_code,valueQuantity_comparator,valueQuantity_value,valueQuantity_unit,valueString,comment,interpretation_display,referenceRange_text,specimen_type,specimen_type_text,container_type,sample_note
20,BRICK_001,2022-06-20 15:12:28,2022-06-20 16:09:38,NaN,Natrium,Natrium,NaN,141.0,mmol/l,NaN,VALAB_V,Normal,136 - 145,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
63,BRICK_002,2022-06-20 14:57:58,2022-06-20 15:47:23,NaN,Natrium,Natrium,NaN,142.0,mmol/l,NaN,VALAB_V,Normal,136 - 145,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
111,BRICK_003,2022-06-20 14:46:36,2022-06-20 15:48:02,NaN,Natrium,Natrium,NaN,138.0,mmol/l,NaN,VALAB_V,Normal,136 - 145,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
152,BRICK_004,2022-07-04 15:42:42,2022-07-04 16:24:37,NaN,Natrium,Natrium,NaN,140.0,mmol/l,NaN,VALAB_V,Normal,136 - 145,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
199,BRICK_005,2022-12-14 10:25:38,2022-12-14 11:19:55,NaN,Natrium,Natrium,NaN,140.0,mmol/l,NaN,VALAB_V,Normal,136 - 145,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
249,BRICK_006,2022-12-14 11:05:20,2022-12-14 12:17:35,NaN,Natrium,Natrium,NaN,140.0,mmol/l,NaN,VALAB_V,Normal,136 - 145,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
289,BRICK_007,2022-07-11 11:19:45,2022-07-11 12:16:02,NaN,Natrium,Natrium,NaN,138.0,mmol/l,NaN,VALAB_V,Normal,136 - 145,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
357,BRICK_008,2022-08-29 10:05:12,2022-08-29 11:07:14,NaN,Natrium,Natrium,NaN,143.0,mmol/l,NaN,VALAB_V,Normal,136 - 145,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
422,BRICK_010,2023-06-05 11:52:04,2023-06-05 12:42:44,NaN,Natrium,Natrium,NaN,137.0,mmol/l,NaN,VALAB_V,Normal,136 - 145,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
460,BRICK_011,2022-07-25 14:28:38,2022-07-25 15:35:54,NaN,Natrium,Natrium,NaN,142.0,mmol/l,NaN,VALAB_V,Normal,136 - 145,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN


In [77]:
lab_data_baseline['code_code_original'] = lab_data_baseline['code_code_original'].fillna('NA')

In [78]:
codes = lab_data_baseline.code_code_original.unique().astype(str)
codes

array(['ORD8', 'ALB_U', 'TE_U', 'TE_U_KR', 'ALB_U_KR', 'KREA_U', 'LORCCA',
       'BRICK', 'LDH', 'SCHWARTZ-Bedsite', 'LENGTH_ORD', 'LENGTH_DAT',
       'TRAF', 'KREA', 'CRP', 'DBIL', 'FERT', 'B12', 'FOLZ', 'K', 'NA',
       'FE', 'BAF9', 'UREU', 'TBIL', 'AFOS', 'GGT', 'ASAT', 'ALAT',
       'VITD', 'MCV', 'RDW', 'GRAT', 'HB', 'HT', 'LEU', 'ERY', 'TROMBO',
       'RETI', 'NRCA', 'ICT', 'LIP', 'HEM', 'TSAT', 'TNFa', 'OSMO_U',
       'GLUC_AO', 'SGEW', 'Eiwit', 'ERY_UR', 'LEUKO', 'Ketonen',
       'Bilirubine', 'Urobilinogeen', 'Nitriet', 'PH', 'UREU_U',
       'AB12_Roche', 'BGL', 'SCR', 'KRBL', 'G6PD', 'PIKI', 'F351',
       'FLAM_opti', 'FKAP_opti', 'FKALA_UR', 'BGK', 'HPLC HbA2',
       'HPLC HbF', 'HPLC HbS', 'BASO_DA', 'EO_DA', 'NEUT_DA', 'LYMFO_DA',
       'MONO_DA', 'HPLC HbA0', 'eDA_Neutro#', 'eDA_IG#', 'eDA_Baso#',
       'eDA_Eo#', 'eDA_Lymfo#', 'eDA_Mono#', 'IG_DA', 'ENDO_dummy', 'PT',
       'INR', 'FBGC', 'APTT', 'CAC', 'CAG', 'SVRIJ', 'SAC', 'STOT', 'F8S',
       'ATRO', '

In [79]:
lab_data_baseline[lab_data_baseline['code_code_original'] == 'nan']

,studieID,effectiveDateTime,issued,code_code_original,code_display_original,hix_code,valueQuantity_comparator,valueQuantity_value,valueQuantity_unit,valueString,comment,interpretation_display,referenceRange_text,specimen_type,specimen_type_text,container_type,sample_note


In [80]:
len(codes)

101

In [81]:
## Notte there is a code BRICK...let's examine

In [82]:
lab_data_baseline[lab_data_baseline.code_code_original =='BRICK'].head(5)

,studieID,effectiveDateTime,issued,code_code_original,code_display_original,hix_code,valueQuantity_comparator,valueQuantity_value,valueQuantity_unit,valueString,comment,interpretation_display,referenceRange_text,specimen_type,specimen_type_text,container_type,sample_note
7,BRICK_001,2022-06-20 15:11:17,2022-06-20 15:22:57,BRICK,BRICK,Trial/Studie,NaN,NaN,NaN,Afgenomen,BRICK,NaN,NaN,Bloed,Studies/trials,EDTA 6 ml,NaN
49,BRICK_002,2022-06-20 14:51:50,2022-06-20 15:07:14,BRICK,BRICK,Trial/Studie,NaN,NaN,NaN,Afgenomen,BRICK,NaN,NaN,Bloed,Studies/trials,EDTA 6 ml,NaN
98,BRICK_003,2022-06-20 14:45:43,2022-06-20 15:07:16,BRICK,BRICK,Trial/Studie,NaN,NaN,NaN,Afgenomen,BRICK,NaN,NaN,Bloed,Studies/trials,EDTA 6 ml,NaN
186,BRICK_004,2022-07-04 15:43:16,2022-07-04 15:50:06,BRICK,BRICK,Trial/Studie,NaN,NaN,NaN,Afgenomen,BRICK,NaN,NaN,Bloed,Studies/trials,EDTA 6 ml,NaN
229,BRICK_005,2022-12-14 10:26:54,2022-12-14 10:37:03,BRICK,BRICK,Trial/Studie,NaN,NaN,NaN,Afgenomen,BRICK,NaN,NaN,Bloed,Studies/trials,EDTA 6 ml,NaN


In [83]:
len(example.columns)

39

Scientists not interested in ICT, LIP, HEM, ORD8, BAF9 or EX_LUMC therefore we will drop these values

In [84]:
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'ICT'].index)
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'LIP'].index)
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'HEM'].index)
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'ORD8'].index)
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'BAF9'].index)
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'EX_LUMC'].index)

Also according to example there are about 50 other codes they do ont care for. We will drop these here too: 

In [85]:
codes0 = [code + "0" for code in codes]
codes0_set = set(codes0)
expected_set = set(example.columns)
expected_set

{'AFOS0',
 'ALAT0',
 'ALB_U0',
 'ALB_U_KR0',
 'ASAT0',
 'B120',
 'CRP0',
 'DBIL0',
 'ERY0',
 'FE0',
 'FERT0',
 'FOLZ0',
 'GGT0',
 'GRAT0',
 'HB0',
 'HT0',
 'K00',
 'KREA0',
 'KREA_U0',
 'LDH0',
 'LEU0',
 'MCV0',
 'NA0',
 'NRCA',
 'Participant Creation Date',
 'Participant Id',
 'Participant Status',
 'RDW0',
 'RETI0',
 'SCHWARTZ_Bedsite0',
 'Site Abbreviation',
 'TBIL0',
 'TE_U0',
 'TE_U_KR0',
 'TRAF0',
 'TROMBO0',
 'TSAT0',
 'UREU0',
 'VITD0'}

In [86]:
# here we will show columsn we were essentially asked to get rid of
droppable = codes0_set - expected_set
list_droppable = list(droppable)
list_droppable = [code[:-1] for code in list_droppable]
#list_droppable

In [87]:
# here we can show what we will keep
list_keepable = expected_set
list_keepable =  [code[:-1] for code in list_keepable]
list_keepable

['ASAT',
 'HB',
 'TE_U_KR',
 'VITD',
 'CRP',
 'SCHWARTZ_Bedsite',
 'HT',
 'ALB_U',
 'NRC',
 'UREU',
 'KREA_U',
 'DBIL',
 'NA',
 'ERY',
 'FOLZ',
 'LDH',
 'MCV',
 'KREA',
 'ALB_U_KR',
 'Participant Creation Dat',
 'TBIL',
 'TROMBO',
 'TRAF',
 'TE_U',
 'LEU',
 'AFOS',
 'Site Abbreviatio',
 'FE',
 'B12',
 'GGT',
 'FERT',
 'ALAT',
 'RDW',
 'K0',
 'Participant I',
 'Participant Statu',
 'TSAT',
 'GRAT',
 'RETI']

In [88]:
list_keepable_amended = ['CRP',
                         'FERT',
                         'TROMBO',
                         'MCV',
                         'K',
                         'FOLZ',
                         'HB',
                         'HT',
                         'UREU',
                         'VITD',
                         'GGT',
                         'ALB_U',
                         'SCHWARTZ-Bedsite',
                         'ASAT',
                         'ERY',
                         'TBIL',
                         'DBIL',
                         'TE_U',
                         'TRAF',
                         'AFOS',
                         'NA',
                         'GRAT',
                         'LDH',
                         'FE',
                         'ALB_U_KR',
                         'B12',
                         'LEU',
                         'RETI',
                         'TSAT',
                         'KREA_U',
                         'ALAT',
                         'KREA',
                         'TE_U_KR',
                         'NRCA',
                         'RDW',]

In [89]:
# lab_data_baseline = lab_data_baseline[~lab_data_baseline[lab_data_baseline['code_code_original'].isin(list_droppable)]]
# lab_data_baseline
lab_data_baseline = lab_data_baseline[lab_data_baseline['code_code_original'].isin(list_keepable_amended)]

In [90]:
lab_data_baseline['code_code_original'].unique()

array(['ALB_U', 'TE_U', 'TE_U_KR', 'ALB_U_KR', 'KREA_U', 'LDH',
       'SCHWARTZ-Bedsite', 'TRAF', 'KREA', 'CRP', 'DBIL', 'FERT', 'B12',
       'FOLZ', 'K', 'NA', 'FE', 'UREU', 'TBIL', 'AFOS', 'GGT', 'ASAT',
       'ALAT', 'VITD', 'MCV', 'RDW', 'GRAT', 'HB', 'HT', 'LEU', 'ERY',
       'TROMBO', 'RETI', 'NRCA', 'TSAT'], dtype=object)

In [91]:
new_codes = list(lab_data_baseline['code_code_original'].unique())

Scientist wants to get rid of doubles, i.e. brick 08, 12, 21 and 25. There are doubles because labs were not taken at precisely the same time 

In [92]:
#example
lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_008']['effectiveDateTime'].unique()

<DatetimeArray>
['2022-08-29 10:05:12', '2022-08-29 10:05:53', '2022-08-29 11:47:51']
Length: 3, dtype: datetime64[ns]

In [93]:
#get the brick8 we want by resetting all the times to be the same
brick_8_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_008']#[lab_data['effectiveDateTime'] == '2022-08-29 10:05:12.0000000']
brick_8_wanted['effectiveDateTime'] = '2022-08-29 10:05:12.0000000'
#brick_8_wanted

C:\Users\Candace\AppData\Local\Temp\2\ipykernel_6296\449678190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brick_8_wanted['effectiveDateTime'] = '2022-08-29 10:05:12.0000000'


In [94]:
#len(brick_8_wanted)

In [95]:
#len(brick_8_wanted.code_code_original.unique())

It did not work the way we wanted because there were repeats. Let's use only one date:

In [96]:
brick_8_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_008'][lab_data_baseline['effectiveDateTime'] == '2022-08-29 10:05:12.0000000']
#brick_8_wanted 

C:\Users\Candace\AppData\Local\Temp\2\ipykernel_6296\3824156078.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  brick_8_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_008'][lab_data_baseline['effectiveDateTime'] == '2022-08-29 10:05:12.0000000']


In [97]:
#12
lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012']['effectiveDateTime'].unique()

<DatetimeArray>
['2022-03-16 15:43:08', '2022-03-16 16:07:41', '2022-07-18 10:02:25']
Length: 3, dtype: datetime64[ns]

In [98]:
# Here we will assume scientist wants the lab values 4 months later dropped, as not a baseline value
brick_12_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'][lab_data_baseline['effectiveDateTime'].isin(['2022-03-16 15:43:08', '2022-03-16 16:07:41'])]
# we will reset to all the same date
#brick_12_wanted 
brick_12_wanted['effectiveDateTime'] = '2022-03-16 15:43:08'

C:\Users\Candace\AppData\Local\Temp\2\ipykernel_6296\2768992246.py:2: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  brick_12_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'][lab_data_baseline['effectiveDateTime'].isin(['2022-03-16 15:43:08', '2022-03-16 16:07:41'])]
C:\Users\Candace\AppData\Local\Temp\2\ipykernel_6296\2768992246.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  brick_12_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'][lab_data_baseline['effectiveDateTime'].isin(['2022-03-16 15:43:08', '2022-03-16 16:07:41'])]


In [99]:
lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_021']['effectiveDateTime'].unique()

<DatetimeArray>
['2023-06-29 08:46:36', '2023-06-29 08:47:57', '2023-06-29 12:41:15']
Length: 3, dtype: datetime64[ns]

In [100]:
#brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 08:47:57']

In [101]:
brick_21_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_021']
#brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 08:46:36']
#brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 08:47:57']
#brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 12:41:15']

In [102]:
brick_21_urine = brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 12:41:15']

In [103]:
brick_21_main = brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 08:47:57']

In [104]:
brick_21_wanted = pd.concat([brick_21_urine, brick_21_main])
brick_21_wanted

,studieID,effectiveDateTime,issued,code_code_original,code_display_original,hix_code,valueQuantity_comparator,valueQuantity_value,valueQuantity_unit,valueString,comment,interpretation_display,referenceRange_text,specimen_type,specimen_type_text,container_type,sample_note
1008,BRICK_021,2023-06-29 12:41:15,2023-06-29 13:44:32,ALB_U,Albumine in urine,M-albu.,NaN,0.0030,g/L,NaN,NaN,Normal,< 0.05,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
1009,BRICK_021,2023-06-29 12:41:15,2023-06-29 13:44:32,TE_U,Tot. eiwit in urine,Tot.eiwit,NaN,0.0600,g/l,NaN,NaN,Normal,< 0.15,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
1010,BRICK_021,2023-06-29 12:41:15,2023-06-29 13:44:32,KREA_U,Kreatinine in urine,Kreatinine,NaN,4.5000,mmol/l,NaN,NaN,Normal,2.0 - 8.0,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
1011,BRICK_021,2023-06-29 12:41:15,2023-06-29 13:44:32,TE_U_KR,Tot.eiwit urine/kreat,Eiwit/kreat,NaN,13.3300,mg/mmol,NaN,NaN,Normal,<15,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
1012,BRICK_021,2023-06-29 12:41:15,2023-06-29 13:44:32,ALB_U_KR,Albumine/Kreatinine ratio,Albu/Kreat,NaN,0.6700,g/mol Kreatinine,NaN,NaN,Normal,<3,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
970,BRICK_021,2023-06-29 08:47:57,2023-06-29 09:21:12,HT,Hematocriet,Hematocriet,NaN,0.2200,L/L,NaN,VALAB_V,Low,0.32 - 0.40,Bloed,Veneus bloed,EDTA 2 ml 18-,NaN
971,BRICK_021,2023-06-29 08:47:57,2023-06-29 09:21:12,HB,Hemoglobine,Hemoglobine,NaN,4.4000,mmol/L,NaN,VALAB_V,Critically low,6.6 - 8.4,Bloed,Veneus bloed,EDTA 2 ml 18-,NaN
972,BRICK_021,2023-06-29 08:47:57,2023-06-29 09:08:22,GRAT,Neutrofielen granulocyten abs.,Neutrofielen abs.,NaN,3.4000,10*9/L,NaN,VALAB_V,NaN,NaN,Bloed,Veneus bloed,EDTA 2 ml 18-,NaN
973,BRICK_021,2023-06-29 08:47:57,2023-06-29 09:21:12,RDW,RDW,RDW,NaN,19.7000,%,NaN,VALAB_V,High,"12,0 - 14,0",Bloed,Veneus bloed,EDTA 2 ml 18-,NaN
974,BRICK_021,2023-06-29 08:47:57,2023-06-29 09:21:12,MCV,MCV,MCV,NaN,82.0000,fL,NaN,VALAB_V,Normal,74 - 88,Bloed,Veneus bloed,EDTA 2 ml 18-,NaN


In [105]:
## here we will "redate" them all to match
####brick_21_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_021']
brick_21_wanted['effectiveDateTime'] = '2023-06-29 08:46:36'
#brick_12_wanted

In [106]:
lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_025']['effectiveDateTime'].unique()

<DatetimeArray>
['2022-09-12 15:04:12', '2022-09-12 15:44:02', '2022-09-12 15:52:42']
Length: 3, dtype: datetime64[ns]

In [107]:
#brick_25_wanted 
brick_25_mcv_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_025'][lab_data_baseline['effectiveDateTime'] == '2022-09-12 15:52:42.0000000']
#brick_25_mcv_wanted

C:\Users\Candace\AppData\Local\Temp\2\ipykernel_6296\4213958660.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  brick_25_mcv_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_025'][lab_data_baseline['effectiveDateTime'] == '2022-09-12 15:52:42.0000000']


In [108]:
brick_25_urine_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_025'][lab_data_baseline['effectiveDateTime'] == '2022-09-12 15:44:02.0000000']
print(brick_25_urine_wanted.shape)
#brick_25_urine_wanted.

(5, 17)


C:\Users\Candace\AppData\Local\Temp\2\ipykernel_6296\1564476141.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  brick_25_urine_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_025'][lab_data_baseline['effectiveDateTime'] == '2022-09-12 15:44:02.0000000']


In [109]:
#get the brick25 we want by concatenating and  resetting all the times to be the same
brick_25_wanted = pd.concat([brick_25_mcv_wanted ,brick_25_urine_wanted ])
print(brick_25_wanted.shape)
brick_25_wanted['effectiveDateTime'] = '2022-09-12 15:52:42'
brick_25_wanted.effectiveDateTime.unique()

(35, 17)


array(['2022-09-12 15:52:42'], dtype=object)

In [110]:
#12
lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012']['effectiveDateTime'].unique()

<DatetimeArray>
['2022-03-16 15:43:08', '2022-03-16 16:07:41', '2022-07-18 10:02:25']
Length: 3, dtype: datetime64[ns]

In [111]:
# Here we will assume scientist wants the lab values 4 months later dropped, as not a baseline value
brick_12_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'][lab_data_baseline['effectiveDateTime'].isin(['2022-03-16 15:43:08', '2022-03-16 16:07:41'])]
# we will reset to all the same date
#brick_12_wanted 
brick_12_wanted['effectiveDateTime'] = '2022-03-16 15:43:08'

C:\Users\Candace\AppData\Local\Temp\2\ipykernel_6296\2768992246.py:2: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  brick_12_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'][lab_data_baseline['effectiveDateTime'].isin(['2022-03-16 15:43:08', '2022-03-16 16:07:41'])]
C:\Users\Candace\AppData\Local\Temp\2\ipykernel_6296\2768992246.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  brick_12_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'][lab_data_baseline['effectiveDateTime'].isin(['2022-03-16 15:43:08', '2022-03-16 16:07:41'])]


In [112]:


#now let's drop all of them, then append back on what we want
lab_data_no_ = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_008'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_021'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_025'].index)
#lab_data = 

In [113]:
lab_data_no_.studieID.unique()

array(['BRICK_001', 'BRICK_002', 'BRICK_003', 'BRICK_004', 'BRICK_005',
       'BRICK_006', 'BRICK_007', 'BRICK_009', 'BRICK_010', 'BRICK_011',
       'BRICK_013', 'BRICK_014', 'BRICK_015', 'BRICK_016', 'BRICK_017',
       'BRICK_018', 'BRICK_019', 'BRICK_024', 'BRICK_026', 'BRICK_027',
       'BRICK_028', 'BRICK_029', 'BRICK_030', 'BRICK_031', 'BRICK_032',
       'BRICK_033', 'BRICK_035', 'BRICK_037', 'BRICK_038', 'BRICK_039',
       'BRICK_040', 'BRICK_041', 'BRICK_042', 'BRICK_044', 'BRICK_047',
       'BRICK_048', 'BRICK_049', 'BRICK_050', 'BRICK_051', 'BRICK_052',
       'BRICK_053', 'BRICK_054', 'BRICK_056', 'BRICK_058'], dtype=object)

In [114]:
#lab_data_baseline

In [115]:
# lab_data2_no_ = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_008'].index)
# lab_data2_no_ = lab_data2_no_.drop(lab_data2_no[lab_data2_no_['studieID'] == 'BRICK_012'].index)
# lab_data2_no_ = lab_data2_no_.drop(lab_data2_no[lab_data2_no_['studieID'] == 'BRICK_021'].index)
# lab_data2_no_ = lab_data2_no_.drop(lab_data2_no[lab_data2_no_['studieID'] == 'BRICK_025'].index)

In [116]:
lab_data = pd.concat([lab_data_no_, brick_8_wanted,  brick_12_wanted, brick_25_wanted, brick_21_wanted])
lab_data.shape

(1508, 17)

In [117]:
list_of_data_frames = []
for code in new_codes:
    prenames = ["code_code_original", code]
    frame_name = "_".join(prenames)
    #print(frame_name)
    frame_name = lab_data[lab_data['code_code_original'] == code]
    frame_name = frame_name.add_suffix(code)
    frame_name = frame_name.rename(columns= {frame_name.columns[0]:'studieID'})
    list_of_data_frames.append(frame_name)
    

In [118]:
len(list_of_data_frames)

35

In [119]:
len(new_codes)

35

Let's look at an example:

In [120]:
list_of_data_frames[0].head(3) 

,studieID,effectiveDateTimeALB_U,issuedALB_U,code_code_originalALB_U,code_display_originalALB_U,hix_codeALB_U,valueQuantity_comparatorALB_U,valueQuantity_valueALB_U,valueQuantity_unitALB_U,valueStringALB_U,commentALB_U,interpretation_displayALB_U,referenceRange_textALB_U,specimen_typeALB_U,specimen_type_textALB_U,container_typeALB_U,sample_noteALB_U
1,BRICK_001,2022-06-20 15:09:28,2022-06-20 16:04:08,ALB_U,Albumine in urine,M-albu.,NaN,0.008,g/L,NaN,NaN,Normal,< 0.05,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
93,BRICK_002,2022-06-20 14:58:46,2022-06-20 15:53:17,ALB_U,Albumine in urine,M-albu.,NaN,0.002,g/L,NaN,NaN,Normal,< 0.05,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
141,BRICK_003,2022-06-20 14:47:20,2022-06-20 15:51:28,ALB_U,Albumine in urine,M-albu.,NaN,0.020,g/L,NaN,NaN,Normal,< 0.05,Urine,Urine midstream,Urinebuis 6.0 ml,NaN


In [121]:
list_of_data_frames[1].head(3) 

,studieID,effectiveDateTimeTE_U,issuedTE_U,code_code_originalTE_U,code_display_originalTE_U,hix_codeTE_U,valueQuantity_comparatorTE_U,valueQuantity_valueTE_U,valueQuantity_unitTE_U,valueStringTE_U,commentTE_U,interpretation_displayTE_U,referenceRange_textTE_U,specimen_typeTE_U,specimen_type_textTE_U,container_typeTE_U,sample_noteTE_U
2,BRICK_001,2022-06-20 15:09:28,2022-06-20 16:04:08,TE_U,Tot. eiwit in urine,Tot.eiwit,NaN,0.07,g/l,NaN,NaN,Normal,< 0.15,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
94,BRICK_002,2022-06-20 14:58:46,2022-06-20 15:53:17,TE_U,Tot. eiwit in urine,Tot.eiwit,NaN,0.05,g/l,NaN,NaN,Normal,< 0.15,Urine,Urine midstream,Urinebuis 6.0 ml,NaN
142,BRICK_003,2022-06-20 14:47:20,2022-06-20 15:51:28,TE_U,Tot. eiwit in urine,Tot.eiwit,NaN,0.09,g/l,NaN,NaN,Normal,< 0.15,Urine,Urine midstream,Urinebuis 6.0 ml,NaN


In [122]:
list_of_data_frames[1].shape

(45, 17)

So far so good, we have a frame with in the example above the DBIL. We then tried to do a big reduction; but it produced repeaters

In [123]:
semi_final_df = reduce(lambda left,right: pd.merge(left,right, on=['studieID'], how='outer'), list_of_data_frames)

In [124]:
semi_final_df.shape

(48, 561)

In [125]:
semi_final_df.head(5)

,studieID,effectiveDateTimeALB_U,issuedALB_U,code_code_originalALB_U,code_display_originalALB_U,hix_codeALB_U,valueQuantity_comparatorALB_U,valueQuantity_valueALB_U,valueQuantity_unitALB_U,valueStringALB_U,...,valueQuantity_valueTSAT,valueQuantity_unitTSAT,valueStringTSAT,commentTSAT,interpretation_displayTSAT,referenceRange_textTSAT,specimen_typeTSAT,specimen_type_textTSAT,container_typeTSAT,sample_noteTSAT
0,BRICK_001,2022-06-20 15:09:28,2022-06-20 16:04:08,ALB_U,Albumine in urine,M-albu.,NaN,0.008,g/L,NaN,...,12.30,%,NaN,NaN,Low,15-45,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
1,BRICK_002,2022-06-20 14:58:46,2022-06-20 15:53:17,ALB_U,Albumine in urine,M-albu.,NaN,0.002,g/L,NaN,...,18.80,%,NaN,NaN,Low,20-45,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
2,BRICK_003,2022-06-20 14:47:20,2022-06-20 15:51:28,ALB_U,Albumine in urine,M-albu.,NaN,0.020,g/L,NaN,...,23.38,%,NaN,NaN,Normal,15-45,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
3,BRICK_004,2022-07-04 16:06:50,2022-07-04 17:31:42,ALB_U,Albumine in urine,M-albu.,NaN,0.012,g/L,NaN,...,83.11,%,NaN,NaN,High,20-45,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN
4,BRICK_005,2022-12-14 11:19:56,2022-12-14 12:30:46,ALB_U,Albumine in urine,M-albu.,NaN,0.002,g/L,NaN,...,17.05,%,NaN,NaN,Low,20-45,Bloed,Veneus bloed,Heparinebuis Barricor 3.0 ml 18-,NaN


In [126]:
for studieID in semi_final_df.studieID:
    print(studieID)

BRICK_001
BRICK_002
BRICK_003
BRICK_004
BRICK_005
BRICK_006
BRICK_007
BRICK_008
BRICK_009
BRICK_010
BRICK_011
BRICK_012
BRICK_013
BRICK_014
BRICK_015
BRICK_016
BRICK_017
BRICK_018
BRICK_019
BRICK_021
BRICK_024
BRICK_025
BRICK_026
BRICK_027
BRICK_028
BRICK_029
BRICK_030
BRICK_031
BRICK_032
BRICK_033
BRICK_035
BRICK_037
BRICK_038
BRICK_039
BRICK_040
BRICK_041
BRICK_042
BRICK_044
BRICK_047
BRICK_048
BRICK_049
BRICK_050
BRICK_051
BRICK_052
BRICK_053
BRICK_054
BRICK_056
BRICK_058


In [127]:
# now we need to change the columns names to those expoected by scientist
for  f in semi_final_df.columns:
    print(f)

studieID
effectiveDateTimeALB_U
issuedALB_U
code_code_originalALB_U
code_display_originalALB_U
hix_codeALB_U
valueQuantity_comparatorALB_U
valueQuantity_valueALB_U
valueQuantity_unitALB_U
valueStringALB_U
commentALB_U
interpretation_displayALB_U
referenceRange_textALB_U
specimen_typeALB_U
specimen_type_textALB_U
container_typeALB_U
sample_noteALB_U
effectiveDateTimeTE_U
issuedTE_U
code_code_originalTE_U
code_display_originalTE_U
hix_codeTE_U
valueQuantity_comparatorTE_U
valueQuantity_valueTE_U
valueQuantity_unitTE_U
valueStringTE_U
commentTE_U
interpretation_displayTE_U
referenceRange_textTE_U
specimen_typeTE_U
specimen_type_textTE_U
container_typeTE_U
sample_noteTE_U
effectiveDateTimeTE_U_KR
issuedTE_U_KR
code_code_originalTE_U_KR
code_display_originalTE_U_KR
hix_codeTE_U_KR
valueQuantity_comparatorTE_U_KR
valueQuantity_valueTE_U_KR
valueQuantity_unitTE_U_KR
valueStringTE_U_KR
commentTE_U_KR
interpretation_displayTE_U_KR
referenceRange_textTE_U_KR
specimen_typeTE_U_KR
specimen_type_te

In [128]:
list_keepable_amended_with_id = ["valueQuantity_value" + code for code in list_keepable_amended]
# [code + "0" for code in codes]
list_keepable_amended_with_id = list_keepable_amended_with_id + ['studieID']
list_keepable_amended_with_id

['valueQuantity_valueCRP',
 'valueQuantity_valueFERT',
 'valueQuantity_valueTROMBO',
 'valueQuantity_valueMCV',
 'valueQuantity_valueK',
 'valueQuantity_valueFOLZ',
 'valueQuantity_valueHB',
 'valueQuantity_valueHT',
 'valueQuantity_valueUREU',
 'valueQuantity_valueVITD',
 'valueQuantity_valueGGT',
 'valueQuantity_valueALB_U',
 'valueQuantity_valueSCHWARTZ-Bedsite',
 'valueQuantity_valueASAT',
 'valueQuantity_valueERY',
 'valueQuantity_valueTBIL',
 'valueQuantity_valueDBIL',
 'valueQuantity_valueTE_U',
 'valueQuantity_valueTRAF',
 'valueQuantity_valueAFOS',
 'valueQuantity_valueNA',
 'valueQuantity_valueGRAT',
 'valueQuantity_valueLDH',
 'valueQuantity_valueFE',
 'valueQuantity_valueALB_U_KR',
 'valueQuantity_valueB12',
 'valueQuantity_valueLEU',
 'valueQuantity_valueRETI',
 'valueQuantity_valueTSAT',
 'valueQuantity_valueKREA_U',
 'valueQuantity_valueALAT',
 'valueQuantity_valueKREA',
 'valueQuantity_valueTE_U_KR',
 'valueQuantity_valueNRCA',
 'valueQuantity_valueRDW',
 'studieID']

In [129]:
final_amend = ['studieID',
               'valueQuantity_valueCRP',
                 'valueQuantity_valueFERT',
                 'valueQuantity_valueTROMBO',
                 'valueQuantity_valueMCV',
                 'valueQuantity_valueK',
                 'valueQuantity_valueFOLZ',
                 'valueQuantity_valueHB',
                 'valueQuantity_valueHT',
                 'valueQuantity_valueUREU',
                 'valueQuantity_valueVITD',
                 'valueQuantity_valueGGT',
                 'valueQuantity_valueALB_U',
                 'valueQuantity_valueSCHWARTZ-Bedsite',
                 'valueQuantity_valueASAT',
                 'valueQuantity_valueERY',
                 'valueQuantity_valueTBIL',
                 'valueQuantity_valueDBIL',
                 'valueQuantity_valueTE_U',
                 'valueQuantity_valueTRAF',
                 'valueQuantity_valueAFOS',
                 'valueQuantity_valueNA',
                 'valueQuantity_valueGRAT',
                 'valueQuantity_valueLDH',
                 'valueQuantity_valueFE',
                 'valueQuantity_valueALB_U_KR',
                 'valueQuantity_valueB12',
                 'valueQuantity_valueLEU',
                 'valueQuantity_valueRETI',
                 'valueQuantity_valueTSAT',
                 'valueQuantity_valueKREA_U',
                 'valueQuantity_valueALAT',
                 'valueQuantity_valueKREA',
                 'valueQuantity_valueTE_U_KR',
                 'valueQuantity_valueNRCA',
                 'valueQuantity_valueRDW', ]

In [130]:
filtered = semi_final_df[final_amend]
filtered

,studieID,valueQuantity_valueCRP,valueQuantity_valueFERT,valueQuantity_valueTROMBO,valueQuantity_valueMCV,valueQuantity_valueK,valueQuantity_valueFOLZ,valueQuantity_valueHB,valueQuantity_valueHT,valueQuantity_valueUREU,...,valueQuantity_valueB12,valueQuantity_valueLEU,valueQuantity_valueRETI,valueQuantity_valueTSAT,valueQuantity_valueKREA_U,valueQuantity_valueALAT,valueQuantity_valueKREA,valueQuantity_valueTE_U_KR,valueQuantity_valueNRCA,valueQuantity_valueRDW
0,BRICK_001,0.8,36.1,561.0,81.0,4.5,5.97,4.4,0.21,2.1,...,307.0,10.1,370.6,12.30,6.0,53.0,41.0,11.67,NaN,24.2
1,BRICK_002,0.8,76.8,168.0,71.0,4.7,38.10,7.6,0.33,2.7,...,543.0,4.4,117.8,18.80,7.6,30.0,63.0,6.58,NaN,14.8
2,BRICK_003,0.6,36.9,417.0,83.0,4.1,45.39,4.4,0.20,3.3,...,322.0,8.8,227.1,23.38,7.5,40.0,43.0,12.00,NaN,16.6
3,BRICK_004,11.0,559.0,181.0,119.0,4.5,45.39,4.1,0.18,5.2,...,373.0,6.2,103.7,83.11,6.6,15.0,43.0,13.64,NaN,15.3
4,BRICK_005,5.2,74.9,140.0,66.0,3.8,20.80,6.2,0.29,3.0,...,362.0,5.2,119.2,17.05,8.9,63.0,40.0,12.36,NaN,16.5
5,BRICK_006,2.2,74.7,407.0,52.0,4.0,18.80,7.2,0.34,3.0,...,455.0,5.9,126.9,15.14,5.8,35.0,52.0,10.34,NaN,18.9
6,BRICK_007,2.2,152.0,279.0,116.0,4.4,19.10,6.4,0.29,4.8,...,1218.0,3.3,62.7,24.94,4.3,21.0,58.0,6.98,NaN,15.4
7,BRICK_008,2.6,112.0,122.0,71.0,4.0,26.10,8.3,0.37,3.9,...,279.0,4.6,162.1,19.25,NaN,27.0,84.0,NaN,NaN,19.1
8,BRICK_009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.3,NaN,NaN,15.05,NaN,NaN
9,BRICK_010,8.7,147.0,422.0,80.0,4.2,8.44,4.1,0.18,2.6,...,229.0,17.6,322.3,17.33,NaN,38.0,27.0,NaN,NaN,23.1


In [133]:
columns_dictionary = {'studieID':'Participant ID',
                      'valueQuantity_valueCRP':'CRP0',  
                      'valueQuantity_valueFERT':'FERT0',
                      'valueQuantity_valueTROMBO':'TROMBO0',
                      'valueQuantity_valueMCV':'MCV0',
                      'valueQuantity_valueK':'K0',
                      'valueQuantity_valueFOLZ':'FOLZ0',
                      'valueQuantity_valueHB':'HB0',  
                      'valueQuantity_valueHT':'HT0',
                      'valueQuantity_valueUREU':'UREU0',  
                      'valueQuantity_valueVITD':'VITD0',
                      'valueQuantity_valueGGT':'GGT0',  
                      'valueQuantity_valueALB_U':'ALB_U0',
                      'valueQuantity_valueSCHWARTZ-Bedsite':'SCHWARTZ-Bedsite0',  
                      'valueQuantity_valueASAT':'ASAT0',
                      'valueQuantity_valueERY':'ERY0',  
                      'valueQuantity_valueTBIL':'TBIL0',
                      'valueQuantity_valueDBIL':'DBIL0',  
                      'valueQuantity_valueTE_U':'TE_U0',
                      'valueQuantity_valueTRAF':'TRAF0',  
                      'valueQuantity_valueNA':'NA0',
                      'valueQuantity_valueAFOS':'AFOS0',
                      'valueQuantity_valueGRAT':'GRAT0',  
                      'valueQuantity_valueLDH':'LDH0',
                      'valueQuantity_valueFE':'FE0',  
                      'valueQuantity_valueALB_U_KR':'ALB_U_KR0',
                      'valueQuantity_valueB12':'B120',  
                      'valueQuantity_valueLEU':'LEU0',
                      'valueQuantity_valueRETI':'RETI0',  
                      'valueQuantity_valueTSAT':'TSAT0',
                      'valueQuantity_valueKREA_U':'KREA_U0',  
                      'valueQuantity_valueALAT':'ALAT0',
                      'valueQuantity_valueKREA':'KREA0',  
                      'valueQuantity_valueTE_U_KR':'TE_U_KR0',
                      'valueQuantity_valueNRCA':'NRCA0',  
                      'valueQuantity_valueRDW':'RDW0'}
filtered = filtered.rename(columns = columns_dictionary)
filtered

,Participant ID,CRP0,FERT0,TROMBO0,MCV0,K0,FOLZ0,HB0,HT0,UREU0,...,B120,LEU0,RETI0,TSAT0,KREA_U0,ALAT0,KREA0,TE_U_KR0,NRCA0,RDW0
0,BRICK_001,0.8,36.1,561.0,81.0,4.5,5.97,4.4,0.21,2.1,...,307.0,10.1,370.6,12.30,6.0,53.0,41.0,11.67,NaN,24.2
1,BRICK_002,0.8,76.8,168.0,71.0,4.7,38.10,7.6,0.33,2.7,...,543.0,4.4,117.8,18.80,7.6,30.0,63.0,6.58,NaN,14.8
2,BRICK_003,0.6,36.9,417.0,83.0,4.1,45.39,4.4,0.20,3.3,...,322.0,8.8,227.1,23.38,7.5,40.0,43.0,12.00,NaN,16.6
3,BRICK_004,11.0,559.0,181.0,119.0,4.5,45.39,4.1,0.18,5.2,...,373.0,6.2,103.7,83.11,6.6,15.0,43.0,13.64,NaN,15.3
4,BRICK_005,5.2,74.9,140.0,66.0,3.8,20.80,6.2,0.29,3.0,...,362.0,5.2,119.2,17.05,8.9,63.0,40.0,12.36,NaN,16.5
5,BRICK_006,2.2,74.7,407.0,52.0,4.0,18.80,7.2,0.34,3.0,...,455.0,5.9,126.9,15.14,5.8,35.0,52.0,10.34,NaN,18.9
6,BRICK_007,2.2,152.0,279.0,116.0,4.4,19.10,6.4,0.29,4.8,...,1218.0,3.3,62.7,24.94,4.3,21.0,58.0,6.98,NaN,15.4
7,BRICK_008,2.6,112.0,122.0,71.0,4.0,26.10,8.3,0.37,3.9,...,279.0,4.6,162.1,19.25,NaN,27.0,84.0,NaN,NaN,19.1
8,BRICK_009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.3,NaN,NaN,15.05,NaN,NaN
9,BRICK_010,8.7,147.0,422.0,80.0,4.2,8.44,4.1,0.18,2.6,...,229.0,17.6,322.3,17.33,NaN,38.0,27.0,NaN,NaN,23.1


In [134]:
filtered.columns

Index(['Participant ID', 'CRP0', 'FERT0', 'TROMBO0', 'MCV0', 'K0', 'FOLZ0',
       'HB0', 'HT0', 'UREU0', 'VITD0', 'GGT0', 'ALB_U0', 'SCHWARTZ-Bedsite0',
       'ASAT0', 'ERY0', 'TBIL0', 'DBIL0', 'TE_U0', 'TRAF0', 'AFOS0', 'NA0',
       'GRAT0', 'LDH0', 'FE0', 'ALB_U_KR0', 'B120', 'LEU0', 'RETI0', 'TSAT0',
       'KREA_U0', 'ALAT0', 'KREA0', 'TE_U_KR0', 'NRCA0', 'RDW0'],
      dtype='object')

In [135]:
filtered['Participant Status']= 'Enrolled'

In [136]:
filtered['Site Abbreviation'] = 'ERA'

No idea where to get Participant Creation Date from ...it isn'tteh same as lab date, so let's leave it out.

In [137]:
filtered.shape

(48, 38)

In [138]:
filtered.to_csv('../../secret_data/castor_with_salt_baseline_labs.csv')